In [1]:
def plot_basic_circles():
    fig, ax = plt.subplots()
    MCtriax = patches.Circle((2,0), radius=1., angle=180.,color='red',lw=2,fill=False,label='triaxial compression')
    MCuni_c = patches.Circle((1,0), radius=1., angle=180.,color='blue',lw=2,fill=False,label='uniaxial compression')
    MCuni_t = patches.Circle((-1,0), radius=1., angle=180.,color='green',lw=2,fill=False,label='uniaxial (ex)tension')
    MCss = patches.Circle((0,0), radius=1., angle=180.,color='orange',lw=2,fill=False,label='simple shear')
    ax.add_patch(MCtriax)
    ax.add_patch(MCuni_c)
    ax.add_patch(MCuni_t)
    ax.add_patch(MCss)
    #ax.annotate('$\\sigma_1$',xy=(si[0],cohesion/10.))
    #ax.annotate('$\\sigma_2$',xy=(si[1],cohesion/10.))
    #ax.annotate('$\\sigma_3$',xy=(si[2],cohesion/10.))
    ax.set(adjustable='box', aspect='equal')
    ax.set_xlabel('$\\sigma_\\mathrm{n}$ / MPa')
    ax.set_ylabel('$\\tau$ / MPa')
    ax.legend(fontsize=12)
    ax.set_xlim(-2,4)
    ax.set_ylim(0,2)
    ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()

In [14]:
def calc_principal_stresses(s):
    eigenValues, eigenVectors = np.linalg.eig(s)
    idx = eigenValues.argsort()[::-1]   
    eigenValues = eigenValues[idx]
    eigenVectors = eigenVectors[:,idx]
    return eigenValues, eigenVectors

In [15]:
def deg_to_rad(angle):
    return angle*np.pi/180.

In [16]:
def strength(sn,c,phi):
    friction = np.tan(deg_to_rad(phi))
    return c + friction*sn

In [17]:
def plot_Mohr_Circle(stress_state,cohesion,friction,transp=1):
    si, si_dir = calc_principal_stresses(stress_state)
    s_n = np.linspace(np.minimum(si[-1],0.),si[0],100)
    
    fig, ax = plt.subplots()
    ax.plot(s_n,strength(s_n,cohesion,friction),label='$\\tau_\\mathrm{f} = c + \\sigma_\\mathrm{n} \\mathrm{tan} \\phi$')
    MC13 = patches.Circle(((si[0]+si[2])/2, 0.), radius=(si[2]-si[0])/2, angle=180.,color='red',fill=False,alpha=transp,lw=2)
    MC12 = patches.Circle(((si[1]+si[0])/2, 0.), radius=(si[0]-si[1])/2, angle=180.,color='orange',fill=False,alpha=transp,lw=2)
    MC23 = patches.Circle(((si[1]+si[2])/2, 0.), radius=(si[1]-si[2])/2, angle=90.,color='green',fill=False,alpha=transp,lw=2)
    ax.add_patch(MC13)
    ax.add_patch(MC12)
    ax.add_patch(MC23)
    ax.annotate('$\\sigma_1$',xy=(si[0],cohesion/10.))
    ax.annotate('$\\sigma_2$',xy=(si[1],cohesion/10.))
    ax.annotate('$\\sigma_3$',xy=(si[2],cohesion/10.))
    ax.set_ylim(0.)
    ax.set(adjustable='box', aspect='equal')
    ax.set_xlabel('$\\sigma_\\mathrm{n}$ / MPa')
    ax.set_ylabel('$\\tau$ / MPa')
    ax.legend()
    fig.tight_layout()
    #plt.show()
    return fig,ax

In [18]:
def transform_widget_to_tensor(stress):
    return np.array([
        [stress.children[0].children[0].value,stress.children[1].children[0].value,stress.children[2].children[0].value],
        [stress.children[1].children[0].value,stress.children[1].children[1].value,stress.children[2].children[1].value],
        [stress.children[2].children[0].value,stress.children[2].children[1].value,stress.children[2].children[2].value]])

In [19]:
items = [widgets.FloatText(value=1.,description='$\\sigma_{xx}$'),
         widgets.FloatText(value=2.,description='$\\sigma_{yy}$'),
        widgets.FloatText(value=3.,description='$\\sigma_{zz}$'),
        widgets.FloatText(value=0.,description='$\\sigma_{xy}$'),
        widgets.FloatText(value=0.,description='$\\sigma_{yz}$'),
        widgets.FloatText(value=0.,description='$\\sigma_{xz}$')]
row1 = widgets.Box([items[0]])
row2 = widgets.Box([items[3],items[1]])
row3 = widgets.Box([items[5],items[4],items[2]])
stress = widgets.VBox([row1,row2,row3])
stress

In [3]:
def plot_interactive_standard():
    @interact(c=widgets.BoundedFloatText(value=0.3,min=0, step=0.1, description='$c$ / MPa'),
          f=widgets.BoundedFloatText(value=30.,min=0, description='$\\phi$ / °'))
    def plot_MC(c,f):
        f,a = plot_Mohr_Circle(transform_widget_to_tensor(stress),c,f)
        plt.show();

In [4]:
def plot_interactive_effective():
    @interact(c=widgets.BoundedFloatText(value=0.3,min=0, step=0.1, description='$c$ / MPa'),
              f=widgets.BoundedFloatText(value=30.,min=0, description='$\\phi$ / °'),
              p=widgets.FloatText(value=0.0, step = 0.1, description='$p$ / MPa'))
    def plot_MC(c,f,p):
        sig = transform_widget_to_tensor(stress)
        f,a = plot_Mohr_Circle(sig,c,f,0.8)
        si, si_dir = calc_principal_stresses(sig)
        MC13e = patches.Circle(((si[0]+si[2])/2-p, 0.), radius=(si[2]-si[0])/2, angle=180.,color='red',fill=False,ls='-.',lw=2)
        MC12e = patches.Circle(((si[1]+si[0])/2-p, 0.), radius=(si[0]-si[1])/2, angle=180.,color='orange',fill=False,ls='-.',lw=2)
        MC23e = patches.Circle(((si[1]+si[2])/2-p, 0.), radius=(si[1]-si[2])/2, angle=90.,color='green',fill=False,ls='-.',lw=2)
        a.add_patch(MC13e)
        a.add_patch(MC12e)
        a.add_patch(MC23e)
        a.annotate("$\\sigma_1'$",xy=(si[0]-p,c/10))
        a.annotate("$\\sigma_2'$",xy=(si[1]-p,c/10))
        a.annotate("$\\sigma_3'$",xy=(si[2]-p,c/10))
        plt.show();

In [5]:
def plot_interactive_failure():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
              f=widgets.BoundedFloatText(value=30.,min=0, description='$\\phi$ / °'),
              sx=widgets.FloatText(value=1.3, step = 0.1, description='$\\sigma_{xx}$ / MPa'),
              sz=widgets.FloatText(value=4.0, step = 0.1, description='$\\sigma_{zz}$ / MPa'),
              sxz=widgets.FloatText(value=0.6, step = 0.1, description='$\\sigma_{xz}$ / MPa'),
              show_failure = widgets.Checkbox(value=False, description='Show failure planes'),
              show_xz = widgets.Checkbox(value=False, description='Show planes of $\\sigma_{zz}$ and $\\sigma_{xx}$'),
              show_princ = widgets.Checkbox(value=False, description='Show principal stress planes')
    )
    def plot_MC_states(f,sx,sz,sxz,show_failure,show_xz,show_princ):
        stress = np.array([[sx,sxz],[sxz,sz]])
        si, si_dir = calc_principal_stresses(stress)
        s_n = np.linspace(np.minimum(si[-1],0.),si[0]+(si[0]-np.minimum(si[-1],0.))/5,100)
        #Mittlere Spannung
        s_m = (si[0]+si[-1])/2.
        #Maximale Schubspannung
        tau_max = (si[0]-si[-1])/2.
        #Kohäsion so, dass aktueller Spannungszustand ein Bruchzustand ist
        c = np.maximum(tau_max/np.cos(deg_to_rad(f)) - s_m * np.tan(deg_to_rad(f)), 0.)

        fig, ax = plt.subplots(figsize=(14,10))
        #Bruchgeraden
        ax.plot(s_n,strength(s_n,c,f),label='$\\tau_\\mathrm{f} = c + \\sigma_\\mathrm{n} \\mathrm{tan} \\phi$')
        ax.plot(s_n,-strength(s_n,c,f))
        #Spannungskreis
        MC = patches.Circle((s_m, 0.), radius=tau_max,color='red',fill=False,lw=2)
        ax.add_patch(MC)
        ax.plot(si[0],0,marker='o',color='blue')
        ax.plot(si[-1],0,marker='o',color='blue')
        ax.annotate(' $\\sigma_1$ ',xy=(si[0],c/10.),color='blue')
        ax.annotate(' $\\sigma_3$ ',xy=(si[-1],c/10.),color='blue')
        #Spannungszustand
        ax.plot(sx,-sxz,marker='o',color='black')
        ax.annotate(' $(\\sigma_{xx},\\sigma_{xz})$ ',xy=(sx,-sxz),color='black')
        ax.plot([s_m,sx],[0,-sxz],color='black',ls=':')
        ax.plot(sz,sxz,marker='o',color='black')
        ax.annotate(' $(\\sigma_{zz},\\sigma_{zx})$ ',xy=(sz,sxz),color='black')
        ax.plot([s_m,sz],[0,sxz],color='black',ls=':')
        #Pol
        if (show_failure or show_xz or show_princ):
            ax.plot(sx,sxz,marker='o',color='black')
            ax.annotate(' $P$ ',xy=(sx,sxz))
        #xx,zz planes
        if (show_xz):
            ax.plot([sx,sz],[sxz,sxz],color='black',ls='--')
            ax.plot([sx,sx],[sxz,-sxz],color='black',ls='--')
        #principal planes
        if (show_princ):
            ax.plot([sx,si[0]],[sxz,0],color='blue',ls='--')
            ax.plot([sx,si[-1]],[sxz,0],color='blue',ls='--')
        #kritischster Zustand
        snf = s_m - tau_max*np.sin(deg_to_rad(f))
        tnf = tau_max*np.cos(deg_to_rad(f))
        ax.plot(snf,tnf,marker='s',color='green')
        ax.annotate(' $(\\sigma^*,\\tau^*)$ ',xy=(snf,tnf),color='green')
        ax.plot(snf,-tnf,marker='s',color='green')
        ax.annotate(' $(\\sigma^*,-\\tau^*)$ ',xy=(snf,-tnf),color='green')
        ax.plot([s_m,snf],[0,tnf],color='green',ls=':')
        angle = patches.Arc((s_m, 0.), width=tau_max/3, height=tau_max/3, theta1=0,theta2=90+f,color='green',ls='-',fill=False,lw=2)
        ax.add_patch(angle)
        ax.annotate('$2\\theta^*$',xy=(s_m,tau_max/3),color='green')
        if (show_failure):
            ax.plot([sx,snf],[sxz,tnf],color='green',ls='--')
            ax.plot([sx,snf],[sxz,-tnf],color='green',ls='--')
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel('$\\sigma_\\mathrm{n}$ / MPa')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel('$\\tau$ / MPa')
        #ax.legend()
        fig.suptitle('Required cohesion $c = %.2f$ MPa' %c)
        fig.tight_layout()
        plt.subplots_adjust(top=0.88)
        plt.show();

In [1]:
def plot_interactive_friction():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
              f=widgets.FloatSlider(value=10.,min=0, max=40, step=1, description='$\\phi_\\mathrm{m}$ / °'),
              sv=widgets.FloatText(value=3.0, step = 0.1, description='$\\sigma_\\mathrm{v}$ / MPa'),
              sh=widgets.FloatText(value=1.0, step = 0.1, description='$\\sigma_\\mathrm{h}$ / MPa')
    )
    def plot_MC_mob(f,sv,sh):
        s1 = np.maximum(sv,sh)
        s3 = np.minimum(sv,sh)
        s_n = np.linspace(np.minimum(s3,0.),s1+(1-np.minimum(s3,0.))/5,100)
        #Mittlere Spannung
        s_m = (s1+s3)/2.
        #Maximale Schubspannung
        tau_max = (s1-s3)/2.
        
        tan_phi_m = np.tan(np.deg2rad(f))
        
        tau_mob = lambda sn: tan_phi_m * sn

        fig, ax = plt.subplots(figsize=(18,10))
        #Mobilisierter Reibungswinkel
        ax.plot(s_n,tau_mob(s_n),label='$\\tau_\\mathrm{m} = \\sigma_\\mathrm{n} \\mathrm{tan} \\phi_\\mathrm{m}$',color='green')
        ax.plot(s_n,-tau_mob(s_n),color='green')
        #ax.plot(s_n,s_n*np.tan(np.deg2rad(40)),lw=0)
        #ax.plot(s_n,-s_n*np.tan(np.deg2rad(40)),lw=0)
        
        angle = patches.Arc((0, 0.), width=0.5, height=0.5, theta1=0,theta2=f,color='green',ls='-',fill=False,lw=2)
        ax.add_patch(angle)
        angle = patches.Arc((0, 0.), width=0.5, height=0.5, theta1=-f,theta2=0,color='green',ls='-',fill=False,lw=2)
        ax.add_patch(angle)
        ax.annotate('$\\theta_\\mathrm{m}$',xy=(0.1,0.1),color='green')
        ax.annotate('$\\theta_\\mathrm{m}$',xy=(0.1,-0.15),color='green')
        #Spannungskreis
        MC = patches.Circle((s_m, 0.), radius=tau_max,color='red',fill=False,lw=2)
        ax.add_patch(MC)
        #Hauptspannungen
        ax.plot(s1,0,marker='o',color='blue')
        ax.plot(s3,0,marker='o',color='blue')
        ax.annotate(' $\\sigma_1$ ',xy=(s1,0.1),color='blue')
        ax.annotate(' $\\sigma_3$ ',xy=(s3,0.1),color='blue')
        #Pol
        ax.plot(sh,0,marker='o',color='blue')
        ax.annotate(' $P$ ',xy=(sh,-0.15),color='blue')
        #Spannungspunkte
        with np.errstate(divide='ignore', invalid='ignore'):
            wurzel = np.sqrt((s_m**2 - (1+tan_phi_m**2)*(s_m**2 - tau_max**2))/(1+tan_phi_m**2)**2)
        s_n1 = s_m/(1+tan_phi_m**2) + wurzel
        s_n2 = s_m/(1+tan_phi_m**2) - wurzel
        ax.plot(s_n1,tau_mob(s_n1),marker='o',color='green')
        ax.plot(s_n1,-tau_mob(s_n1),marker='o',color='green')
        ax.plot(s_n2,tau_mob(s_n2),marker='o',color='green')
        ax.plot(s_n2,-tau_mob(s_n2),marker='o',color='green')
        #Wirkungsebenen
        ax.plot([sh,s_n1],[0,tau_mob(s_n1)],color='blue',ls='--')
        ax.plot([sh,s_n1],[0,-tau_mob(s_n1)],color='blue',ls='--')
        ax.plot([sh,s_n2],[0,tau_mob(s_n2)],color='blue',ls='--')
        ax.plot([sh,s_n2],[0,-tau_mob(s_n2)],color='blue',ls='--')
        #layout
        ax.spines['bottom'].set_position('zero')
        ax.spines['left'].set_position('zero')
        ax.set(adjustable='box', aspect='equal')
        ax.set_xlabel('$\\sigma_\\mathrm{n}$ / MPa')
        ax.xaxis.set_label_coords(1.1, 0.5)
        ax.set_ylabel('$\\tau$ / MPa')
        fig.legend()
        fig.tight_layout()
        plt.show();